In [ ]:
#  Install dependencies
!pip install -q bitsandbytes accelerate peft transformers datasets gradio huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.2 MB/s eta 0:00:00


In [ ]:
# ✅ Hugging Face login
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#  Load base model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token  # Important for LLaMA

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Load dataset
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": "/content/drive/MyDrive/Model-2/MedDialog.json"})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
#  Preprocess dataset
def preprocess_function(example):
    flat_utterances = [utt for sublist in example["utterances"] for utt in sublist] if isinstance(example["utterances"][0], list) else example["utterances"]
    full_text = " ".join(flat_utterances)
    return tokenizer(full_text, truncation=True, padding=True, max_length=512, return_tensors="pt")

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

In [ ]:
#  Train/Validation Split
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [ ]:

#  Load model with quantization + LoRA

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import init_empty_weights, infer_auto_device_map
import torch
from peft import LoraConfig, get_peft_model, TaskType

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    # llm_int8_enable_fp32_cpu_offload=True,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True,
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# Data collator
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora-llama2",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=8,
    learning_rate=2e-5,         # Changed from 2e-4  to 2e-5
    fp16=True,
    optim="paged_adamw_32bit",
# --- Evaluation and Saving Strategy
    eval_strategy="epoch",               #  Evaluate at end of each epoch
    save_strategy="epoch",               # Save a checkpoint at the end of each epoch
    save_total_limit=1,                  # Only keep the best checkpoint
    load_best_model_at_end=True,         # Load the best model when training is finished
    metric_for_best_model="eval_loss",   # Use validation loss to determine the best model
    greater_is_better=False,             # Lower validation loss is better
# --- Scheduler and Logging ---

    lr_scheduler_type="cosine",       # Use a cosine learning rate scheduler
    warmup_ratio=0.03,                # 3% of steps are used for warmup
    logging_strategy="steps",         # Log metrics at fixed step intervals
    logging_steps=25,                 # Log every 25 steps
    report_to="tensorboard",          # Enable TensorBoard logging
    logging_dir="./logs",             # Directory for logs

    remove_unused_columns=False,
)


In [ ]:
#  Define Trainer with eval dataset
from transformers import Trainer
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,          #  Include validation set
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] # Stop if eval_loss doesn't improve for 3 epochs
)

#  Train the model
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,2.648786
2,No log,2.647565
3,No log,2.645788
4,No log,2.643990
5,No log,2.642573
6,No log,2.641841
7,No log,2.641424
8,No log,2.640921


TrainOutput(global_step=8, training_loss=2.7140800952911377, metrics={'train_runtime': 23.7565, 'train_samples_per_second': 0.337, 'train_steps_per_second': 0.337, 'total_flos': 162485155921920.0, 'train_loss': 2.7140800952911377, 'epoch': 8.0})

In [7]:
# Save Fine-tuned Adapter

model.save_pretrained("/content/drive/MyDrive/Model-2/Tunned_Model")
tokenizer.save_pretrained("/content/drive/MyDrive/Model-2/Tunned_Model")



# Reload the model for use

In [1]:
! pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [2]:
#  Hugging Face login
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# --- 1. Define Model and Adapter Paths ---
base_model_name = "meta-llama/Llama-2-7b-chat-hf"
adapter_path = "/content/drive/MyDrive/Model-2/Tunned_Model"

# --- 2. Configure Quantization ---
# Load in 4-bit precision for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# --- 3. Load the Base Model ---
print("Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto", # Automatically place the model on the available GPU
    trust_remote_code=True,
)

# --- 4. Load the Fine-Tuned PEFT Adapter ---
# This merges your fine-tuned weights into the base model
print("Loading PEFT adapter...")
model = PeftModel.from_pretrained(base_model, adapter_path)

# --- 5. Load the Tokenizer ---
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# --- 6. Set Model to Evaluation Mode ---
# This is important for inference
model.eval()
print("Model is ready for inference!")

# --- 7. Generate a Response ---
prompt = "I have a persistent cough and a slight fever. What should I consider doing?"
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

print("\nGenerating response...")
with torch.no_grad():
    output = model.generate(
        input_ids=input_ids,
        max_new_tokens=200,
        do_sample=True,
        top_p=0.9,
        temperature=0.7
    )




Loading base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading PEFT adapter...
Loading tokenizer...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Model is ready for inference!

Generating response...


In [6]:
# Decode and print the generated text
response = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
print(f"\nModel Response:\n{response}")


Model Response:
I have a persistent cough and a slight fever. What should I consider doing?

You should consider the following when you have a persistent cough and a slight fever:

1. Stay hydrated: Drink plenty of fluids, such as water, tea, or soup, to help loosen mucus and soothe a sore throat.
2. Rest: Get plenty of rest to help your body fight off the infection.
3. Use over-the-counter medications: Cough suppressants, decongestants, and pain relievers can help relieve symptoms.
4. Try steam inhalation: Breathing in steam from a bowl of hot water or a steam humidifier can help loosen mucus and ease a cough.
5. Avoid irritants: Stay away from smoke, dust, and other irritants that can aggravate a cough and fever.
6. Seek medical attention: If your


# Real One Below

# Model Training Report
## Project Title: Fine-Tuning LLaMA2 with PEFT (LoRA)

### Model and Dataset Overview:
- **Base Model:** LLaMA2 (Causal Language Model)
- **PEFT Strategy:** LoRA (Low-Rank Adaptation)
- **Model Wrapper:** PeftModelForCausalLM from Hugging Face + PEFT
- **Train Set Size:** 1024
- **Data Type:** Instruction-style text prompts
- **Tokenization:** Using LLaMA tokenizer with padding & truncation

### Training Configuration:
- **Trainer API:** Hugging Face Trainer
- **Epochs:** 8
- **Batch Size:** 2
- **Gradient Accumulation Steps:** 8
- **Learning Rate:** 2e−5
- **Optimizer:** paged_adamw_32bit
- **Scheduler:** Cosine learning rate scheduler with a warmup ratio of 0.03.
- **Mixed Precision:** Enabled (fp16=True)
- **Evaluation Strategy:** Per epoch
- **Save Strategy:** Per epoch, keeping only the best model based on validation loss.
- **Logging Strategy:** Logging every 25 steps to TensorBoard.
- **Model Saving Path:** ./lora-llama2

### Training & Evaluation Results:
The provided training logs show the validation loss decreasing consistently over 8 epochs, but the training loss is not available in the logs.

| Epoch | Training Loss | Validation Loss |
|-------|---------------|-----------------|
| 1     | No log        | 2.648786        |
| 2     | No log        | 2.647565        |
| 3     | No log        | 2.645788        |
| 4     | No log        | 2.643990        |
| 5     | No log        | 2.642573        |
| 6     | No log        | 2.641841        |
| 7     | No log        | 2.641424        |
| 8     | No log        | 2.640921        |

**Final Training Loss:** No log available.  
**Final Validation Loss:** ≈2.6409 (at epoch 8)

### Observations & Improvements:
- **Consistent Convergence:** The validation loss steadily decreases from epoch 1 to epoch 8, indicating that the model is learning from the dataset and converging well.
- **Learning Rate and Scheduler:** The use of a lower learning rate (2e−5) and a cosine learning rate scheduler is a good practice for stable fine-tuning, preventing the model from overshooting the optimal parameters.
- **No Training Loss:** The provided data lacks training loss metrics, which is crucial for monitoring for overfitting. If the training loss were to drop significantly while the validation loss plateaus or increases, it would be a clear sign of overfitting.
- **Suggested Improvement:** To get a complete picture, the logging should be adjusted to display the training loss. The `logging_strategy` is set to `"steps"`, but the provided table only shows epoch-based metrics. Ensuring the training loss is logged and monitored would provide valuable insights into the model's performance and help confirm whether it is truly converging or starting to overfit.
